In [1]:
import Pkg

# Create a local project folder in your HOME directory
Pkg.activate(joinpath(homedir(), ".julia", "local_fuzzy_env"))
Pkg.instantiate()

  Activating project at `~/.julia/local_fuzzy_env`


In [2]:
import Pkg
Pkg.add(["JLD2", "DataFrames", "Statistics", "Distributions", "Plots", "FileIO"])

┌ Warning: attempting to remove probably stale pidfile
│   path = "/home/.julia-depot/registries/.pid"
└ @ FileWatching.Pidfile /opt/julia-1.12.2/share/julia/stdlib/v1.12/FileWatching/src/pidfile.jl:300
    Updating registry at `~/.julia-depot/registries/General`
┌ Info: The General registry is installed via git. Consider reinstalling it via
│ the newer faster direct from tarball format by running:
│   pkg> registry rm General; registry add General
└ 
    Updating git-repo `https://github.com/JuliaRegistries/General.git`
   Resolving package versions...
     Project No packages added to or removed from `~/.julia/local_fuzzy_env/Project.toml`
    Manifest No packages added to or removed from `~/.julia/local_fuzzy_env/Manifest.toml`


In [3]:
##############################
# 1. ENVIRONMENT & IMPORTS
##############################

using JLD2
using DataFrames
using Statistics
using Plots

# Load your custom project modules
include("./src/signals/decision_rules.jl")
include("./src/signals/signal_generator.jl")

using .DecisionRules
using .SignalGenerator

println("Environment loaded.")


[ Info: Precompiling GRIJuliaExt [84369c5d-ffb2-5a92-8288-3470980d96d0] (cache misses: wrong dep version loaded (2))

SYSTEM: caught exception of type :MethodError while trying to print a failed Task notice; giving up
[ Info: Precompiling IJuliaExt [2f4121a4-3b3a-5ce6-9c5e-1f2673ce168a] (cache misses: wrong dep version loaded (2))

SYSTEM: caught exception of type :MethodError while trying to print a failed Task notice; giving up


Environment loaded.


In [5]:
##############################
# 2. LOAD JLD2 SPY DATA
##############################

filepath = "spy_clean_valid_iv.jld2"

raw = load(filepath)

# Detect DataFrame inside JLD2 dictionary
df =
    if "df_clean" ∈ keys(raw)
        raw["df_clean"]
    elseif "data" ∈ keys(raw)
        raw["data"]
    else
        error("Could not find DataFrame inside JLD2 file.")
    end

println("Rows loaded: ", size(df, 1))
println("Columns: ", names(df))


Rows loaded: 3926255
Columns: ["Column1", "level_0", "QUOTE_UNIXTIME", "QUOTE_READTIME", "QUOTE_DATE", "QUOTE_TIME_HOURS", "UNDERLYING_LAST", "EXPIRE_DATE", "EXPIRE_UNIX", "DTE", "C_DELTA", "C_GAMMA", "C_VEGA", "C_THETA", "C_RHO", "C_IV", "C_VOLUME", "C_LAST", "C_SIZE", "C_BID", "C_ASK", "STRIKE", "P_BID", "P_ASK", "P_SIZE", "P_LAST", "P_DELTA", "P_GAMMA", "P_VEGA", "P_THETA", "P_RHO", "P_IV", "P_VOLUME", "STRIKE_DISTANCE", "STRIKE_DISTANCE_PCT", "r"]


In [6]:
##############################
# 3. SELECT FEATURES
##############################

selected_cols = [
    :UNDERLYING_LAST,
    :P_IV,
    :r,
    :DTE,
    :C_DELTA,
    :C_GAMMA,
    :C_VEGA,
    :P_LAST
]

df_clean = dropmissing(df[:, selected_cols])

println(first(df_clean, 5))
println("Rows after cleaning: ", size(df_clean, 1))


5×8 DataFrame
 Row │ UNDERLYING_LAST  P_IV     r        DTE      C_DELTA  C_GAMMA  C_VEGA   P_LAST  
     │ Float64          Float64  Float64  Float64  Float64  Float64  Float64  Float64 
─────┼────────────────────────────────────────────────────────────────────────────────
   1 │          366.62  0.24586     3.46      0.0  0.996    0.0033   0.00208     0.01
   2 │          366.62  0.19937     3.46      0.0  0.99543  0.00479  0.00259     0.01
   3 │          366.62  0.17624     3.46      0.0  0.99052  0.00887  0.0041      0.01
   4 │          366.62  0.16372     3.46      0.0  0.02337  0.04667  0.00869     2.45
   5 │          366.62  0.1974      3.46      0.0  0.01828  0.02773  0.00698     3.89
Rows after cleaning: 3926255


In [ ]:
##############################
# 4. CREATE FUZZY SIGNALS
##############################

function run_fuzzy_pipeline(row)
    fs = FuzzySignal(
        row.UNDERLYING_LAST,
        row.P_IV,
        row.r
    )
    memberships = evaluate_rules(fs)
    crispsig = crisp_signal(fs)
    return (memberships, crispsig)
end

results = [run_fuzzy_pipeline(row) for row in eachrow(df_clean)]

df_clean.memberships = [r[1] for r in results]
df_clean.signal = [r[2] for r in results]

println(first(df_clean, 5))


In [ ]:
##############################
# 5. VISUALIZATION
##############################

signal_to_int = Dict("StrongSell"=>1, "Sell"=>2, "Hold"=>3, "Buy"=>4, "StrongBuy"=>5)

df_clean.signal_int = [signal_to_int[s] for s in df_clean.signal]

plot(df_clean.signal_int,
    title="Fuzzy Trading Signal Over Time",
    xlabel="Index",
    ylabel="Signal (1=Sell → 5=Buy)",
    legend=false
)


In [ ]:
##############################
# 6. CALIBRATION
##############################

function calibrate_threshold(values; bins=5)
    qs = quantile(values, range(0, 1; length=bins+1))
    return qs
end

iv_bins = calibrate_threshold(df_clean.P_IV)
price_bins = calibrate_threshold(df_clean.UNDERLYING_LAST)
r_bins = calibrate_threshold(skipmissing(df_clean.r))

println("IV fuzzy bin cutoffs: ", iv_bins)
println("Price fuzzy bin cutoffs: ", price_bins)
println("Interest rate fuzzy bin cutoffs: ", r_bins)


In [ ]:
##############################
# 7. SIGNAL EVALUATION
##############################

df_clean.return = [i==size(df_clean,1) ? missing :
                    (df_clean.UNDERLYING_LAST[i+1] - df_clean.UNDERLYING_LAST[i]) /
                     df_clean.UNDERLYING_LAST[i]
                   for i in 1:size(df_clean,1)]

group_stats = combine(groupby(df_clean, :signal),
    :return => mean => :avg_return,
    :return => std => :volatility
)

println(group_stats)


In [ ]:
##############################
# 8. SUMMARY
##############################

println("Final fuzzy signal distribution:")
println(combine(groupby(df_clean, :signal), nrow => :count))
